In [1]:
#Imports
import time
import pandas as pd
import json
from bs4 import BeautifulSoup as soup
from tqdm import trange
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException 
import ipywidgets as widgets
import sys
import numpy as np

Run the next cell to get the urls of all the main leagues with data available

In [9]:
##### Get league links
def getLeagueLinks(main_url):
    
    driver = webdriver.Chrome('chromedriver.exe')
    driver.minimize_window()
        
    main = driver.get(main_url)
    leagues = []
    for i in range(22):
        league = driver.find_element_by_xpath('//*[@id="popular-tournaments-list"]/li['+str(i+1)+']/a').get_attribute('href')
        leagues.append(league)
    driver.close()
    return leagues
main_url = 'https://www.whoscored.com'
leagues = getLeagueLinks(main_url)
for i in range(len(leagues)):
    print('League index : '+str(i)+' , League name : '+leagues[i])

League index : 0 , League name : https://www.whoscored.com/Regions/252/Tournaments/2/England-Premier-League
League index : 1 , League name : https://www.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A
League index : 2 , League name : https://www.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga
League index : 3 , League name : https://www.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga
League index : 4 , League name : https://www.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1
League index : 5 , League name : https://www.whoscored.com/Regions/177/Tournaments/21/Portugal-Liga-NOS
League index : 6 , League name : https://www.whoscored.com/Regions/155/Tournaments/13/Netherlands-Eredivisie
League index : 7 , League name : https://www.whoscored.com/Regions/182/Tournaments/77/Russia-Premier-League
League index : 8 , League name : https://www.whoscored.com/Regions/31/Tournaments/95/Brazil-Brasileir%C3%A3o
League index : 9 , League name : https://www.whoscored.com/Reg

Now, select a league and season. Keep in mind that there might not be data available for all leagues for all seasons. You will have to check.

In [83]:
###### Choose League and season
Leagues = ['EPL','SA','LL','BuLi','L1','LigaNOS','Eredivisie','RPL','Brazil','MLS','Turkey',
          'Championship','Premiership','LeagueOne','LeagueTwo','Argentina','Belgium','China','BuliII',
          'UCL','UEL','NationsLeagueA']
seasons = ['202021','201920','201819','201718','201617','201516','201415','201314','201213','201112','201011','200910']

League = widgets.Dropdown(
    options=Leagues,
    value=Leagues[0],
    description='League',
    disabled=False,
)
Season = widgets.Dropdown(
    options=seasons,
    value=seasons[0],
    description='Season',
    disabled=False,
)
box = widgets.HBox(children = [League,Season])
display(box)

In [165]:
###### Display the Chosen values for verification
League.value,Season.value

('LL', '201617')

Run the following cell, wait for the execution to finish, then visit the chrome tab and check if there is an ad blocking the view. If so, click on the X to close the ad and come back to the notebook. If there are no ads, just come back to the notebook.

In [166]:
league_index = Leagues.index(League.value)
season_index = seasons.index(Season.value)+1
comp_url = leagues[league_index]
driver = webdriver.Chrome('chromedriver.exe')
comp = driver.get(comp_url)
xpathstring = '//*[@id="seasons"]/option['+str(season_index)+']'
season = driver.find_element_by_xpath(xpathstring).click()
time.sleep(5)

Run the following cells one by one

In [167]:
fixtures_page = driver.find_element_by_xpath('//*[@id="link-fixtures"]').click()
time.sleep(5)

In [168]:
date_config_btn = driver.find_element_by_xpath('//*[@id="date-config-toggle-button"]').click()
time.sleep(5)

In [169]:
year1 = driver.find_element_by_xpath('//*[@id="date-config"]/div[1]/div/table/tbody/tr/td[1]/div/table/tbody/tr[1]/td').click()
selectable_months = driver.find_element_by_xpath('//*[@id="date-config"]/div[1]/div/table/tbody/tr/td[2]/div/table').find_elements_by_class_name("selectable")

n_months = len(selectable_months)

year2 = driver.find_element_by_xpath('//*[@id="date-config"]/div[1]/div/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td').click()
selectable_months = driver.find_element_by_xpath('//*[@id="date-config"]/div[1]/div/table/tbody/tr/td[2]/div/table').find_elements_by_class_name("selectable")

n_months += len(selectable_months)
date_config_btn = driver.find_element_by_xpath('//*[@id="date-config-toggle-button"]').click()

In [170]:
n_months

10

In [171]:
#for month_element in selectable_months:
match_links = []
def check_exists(row):
    try:
        elem = row.find("a", {"class":"result-1 rc"})
        if elem is None:
            return False
    except NoSuchElementException:
        return False
    return True
for i in range(n_months):
    time.sleep(2)
    fixtures_table = driver.find_element_by_xpath('//*[@id="tournament-fixture"]')
    fixtures_table = fixtures_table.get_attribute('innerHTML')
    fixtures_table = soup(fixtures_table, features="lxml")
    table_rows1 = fixtures_table.find_all("div", {"class":"divtable-row col12-lg-12 col12-m-12 col12-s-12 col12-xs-12"})    
    table_rows2 = fixtures_table.find_all("div", {"class":"divtable-row col12-lg-12 col12-m-12 col12-s-12 col12-xs-12 alt"})
    table_rows = table_rows1+table_rows2
    links = []
    links = [main_url+row.find("a", {"class":"result-1 rc"}).get("href") for row in table_rows if check_exists(row)]
    for link in links:
        match_links.append(link)
    previous_month = driver.find_element_by_xpath('//*[@id="date-controller"]/a[1]').click()
if len(match_links) != 380:
    fixtures_table = driver.find_element_by_xpath('//*[@id="tournament-fixture"]')
    fixtures_table = fixtures_table.get_attribute('innerHTML')
    fixtures_table = soup(fixtures_table)
    table_rows1 = fixtures_table.find_all("div", {"class":"divtable-row col12-lg-12 col12-m-12 col12-s-12 col12-xs-12"})    
    table_rows2 = fixtures_table.find_all("div", {"class":"divtable-row col12-lg-12 col12-m-12 col12-s-12 col12-xs-12 alt"})
    table_rows = table_rows1+table_rows2
    links = []
    links = [main_url+row.find("a", {"class":"result-1 rc"}).get("href") for row in table_rows if check_exists(row)]
    for link in links:
        match_links.append(link)

match_links = list(dict.fromkeys(match_links))
driver.close()

In [172]:
len(match_links)

380

In [173]:
match_links = [[match_links[i]] for i in range(len(match_links))]

In [ ]:
# match_links

Save the list of urls as a csv to be used for the actual match-scraping

In [174]:
import csv
filename = League.value+Season.value+'.csv'
with open(filename, 'w') as f: 
    # using csv.writer method from CSV package 
    write = csv.writer(f) 
    write.writerow(['col1']) 
    write.writerows(match_links) 